In [20]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import chromadb
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

load_dotenv()

model = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_chroma import Chroma
os.environ["LANGSMITH_PROJECT"] = "data-chunking-pdf"

chroma_client = chromadb.HttpClient(host='localhost', port=8000)
print(chroma_client.heartbeat())

vector_store = Chroma(
    collection_name="llm-rag-ai",
    embedding_function=embeddings,
    client=chroma_client
)
retriever = vector_store.as_retriever()


1739368798506000275


In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """
**Context**: {context}
You are an AI assistant specialized in answering questions about Artificial Intelligence (AI), Large Language Models (LLMs), Retrieval-Augmented Generation (RAG), and Python programming. Your task is to provide clear, accurate, and concise explanations. If the question is outside your scope, politely inform the user.

**User Question**: {question}

**Your Response**:
1. Provide a clear and structured answer.
2. Include examples, code snippets, or analogies if applicable.
3. Do not include any additional explanations or context.
4. Do not hallucinate.
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

In [16]:
question = "provide me a good example of rag applied to real-state"

In [17]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
#
# rag_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt_template
#         | llm
#         | StrOutputParser()
# )
#
# response = rag_chain.invoke(question)
# print(response)

In [18]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=3)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [19]:
response = graph.invoke({"question": question})
print(response["answer"])

A good example of Retrieval-Augmented Generation (RAG) applied to real estate is in generating property valuations and recommendations. Here's how it works:

1. **Data Collection**: The RAG model collects and retrieves data from various sources such as historical housing prices, local market trends, demographic information, and economic indicators.

2. **Analysis and Generation**: The model analyzes this data using machine learning algorithms to understand patterns and trends. It then generates accurate property valuations and recommendations.

3. **Personalized Recommendations**: Real estate agents can input specific property details or client preferences into the RAG model, which will generate tailored recommendations and valuations. This helps agents make informed decisions and provide better service to clients.

This application enhances the decision-making process in real estate by providing data-driven insights and reducing the reliance on subjective judgment.
